In [ ]:
!pip uninstall -y typing # this should avoid  AttributeError: type object 'Callable' has no attribute '_abc_registry'

!pip install  "git+https://github.com/dreamquark-ai/tabnet.git@develop#egg=pytorch_tabnet" --upgrade

In [8]:
import warnings
warnings.filterwarnings('ignore')

from pytorch_tabnet.multitask import TabNetMultiTaskClassifier

import torch
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import random
np.random.seed(0)

from tqdm.notebook import tqdm

import os

from matplotlib import pyplot as plt
%matplotlib inline

In [9]:
d = "C:\kaggle_data\credit_card"
lst = os.listdir(d)
print(lst)
train = pd.read_csv(d + '\\' +lst[3])
test = pd.read_csv(d + '\\' +lst[2])
ss = pd.read_csv(d + '\\' +lst[1])
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

['.ipynb_checkpoints', 'sample_submission.csv', 'test.csv', 'train.csv', 'Untitled.ipynb']


In [10]:
n = 5

In [11]:
random.seed(42)
skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=42)

In [7]:
def tr(train):
    folds=[]
    losses=[]
    for train_idx, valid_idx in skf.split(train, train['credit']):
        folds.append((train_idx, valid_idx))



    models={}
    for fold in range(n):
        max_epochs = 1000

        train_idx, valid_idx = folds[fold]
        X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                             train['credit'][train_idx].values, train['credit'][valid_idx].values 
        clf = TabNetMultiTaskClassifier(n_steps=1,
                                    cat_idxs=cat_idxs,
                                    cat_dims=cat_dims,
                                    cat_emb_dim=1,
                                    optimizer_fn=torch.optim.Adam,
                                    optimizer_params=dict(lr=2e-2),
                                    scheduler_params={"step_size":50,
                                                      "gamma":0.9},
                                    scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                    mask_type='entmax', 
                                    lambda_sparse=0, 
                           )

        clf.fit(
            X_train=X_train, y_train=y_train,
            max_epochs=max_epochs ,
            patience=30,
            batch_size=1024,
            virtual_batch_size=128,
            num_workers=1,
            drop_last=False,
            eval_set=[(X_valid, y_valid)],
            eval_metric=["logloss"])

        models[fold]=clf



        losses.append(log_loss(y_valid, clf.predict_proba(X_valid)))
    ls = sum(losses)/n
    print(ls)
    return models